In [1]:
import pandas as pd 
import numpy as np
import seaborn as sb
from scipy import stats
import matplotlib.pyplot as plt
import math
from xgboost import XGBRegressor,XGBClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [2]:
# Already have asteroid dataset in drive. Downloading it from drive
!gdown --id 1-afEh_SyLdQh1tFH3mlcCgXLPj4FWSKm

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-afEh_SyLdQh1tFH3mlcCgXLPj4FWSKm
To: /content/Cleaned_Asteroid.csv
100% 174M/174M [00:02<00:00, 72.0MB/s]


In [3]:
df=pd.read_csv("Cleaned_Asteroid.csv")

In [4]:
df=df.astype({'diameter':float,'condition_code':int})
df.head(5)

,full_name,a,e,i,om,w,q,ad,per_y,data_arc,condition_code,n_obs_used,H,diameter,albedo,neo,pha,moid,diam_bin
0,1 Ceres,2.769165,0.076009,10.594067,80.305532,73.597694,2.558684,2.979647,4.608202,8822.0,0,1002,3.34,939.400,0.0900,N,N,1.59478,Very Large
1,2 Pallas,2.772466,0.230337,34.836234,173.080063,310.048857,2.133865,3.411067,4.616444,72318.0,0,8490,4.13,545.000,0.1010,N,N,1.23324,Very Large
2,3 Juno,2.669150,0.256942,12.988919,169.852760,248.138626,1.983332,3.354967,4.360814,72684.0,0,7104,5.33,246.596,0.2140,N,N,1.03454,Large
3,4 Vesta,2.361418,0.088721,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,24288.0,0,9325,3.20,525.400,0.4228,N,N,1.13948,Very Large
4,5 Astraea,2.574249,0.191095,5.366988,141.576604,358.687608,2.082324,3.066174,4.130323,63431.0,0,2861,6.85,106.699,0.2740,N,N,1.09589,Large


### Feature Selection

#### We are using Correlation Matrix to select Most Important Feature

In [ ]:
corr=df.corr()
corr['diameter'].abs().sort_values(ascending=False)

diameter          1.000000
H                 0.565614
data_arc          0.492110
n_obs_used        0.386038
moid              0.332416
q                 0.329698
a                 0.144748
albedo            0.105658
ad                0.093440
condition_code    0.073546
i                 0.052540
e                 0.049107
per_y             0.048955
w                 0.002980
om                0.001155
Name: diameter, dtype: float64

After performing correlation matrix, it is found that H,data_arc,n_obs_used,moid,q,a,albedo have higher correlation with matrix.

Select the features with higher absolute correlation value.

In [5]:
df=df[['H','data_arc','n_obs_used','moid','q','a','albedo','neo','pha','diameter']]

## Splitting the dataset

* Since the diameter has missing values, we will have missing values of diameter in test data set and train dataset will have non-missing values.

* We can also perform Validation Split for better convenience

In [6]:
# Test Data of missing values of diameter
te_dat = df[df["diameter"].isna()]
df.dropna(inplace=True)

ytr = df["diameter"]
xtr = df.drop("diameter", axis=1)
xte = te_dat.drop("diameter", axis=1)

# 84-16 Splitting With no Random state
print(xtr.shape,ytr.shape,xte.shape)

(137681, 9) (137681,) (702055, 9)


## Encoding the data

* For Categorical Features

In [7]:
# NEO
xtr_neo_encode=pd.get_dummies(xtr['neo'], drop_first=True)
xte_neo_encode=pd.get_dummies(xte['neo'], drop_first=True)

In [8]:
# PHA
xtr_pha_encode=pd.get_dummies(xtr['pha'], drop_first=True)
xte_pha_encode=pd.get_dummies(xte['pha'], drop_first=True)

* For Numerical Features

In [9]:
sd=StandardScaler()

#### Train Data

In [10]:
sd.fit(xtr[['a', 'q', 'data_arc', 'n_obs_used', 'H',
       'albedo', 'moid']])
xtr_norm=sd.transform(xtr[['a', 'q', 'data_arc', 'n_obs_used', 'H',
       'albedo', 'moid']])
xtr_norm=pd.DataFrame(data=xtr_norm,columns=['a', 'q', 'data_arc', 'n_obs_used', 'H',
       'albedo', 'moid'])
xtr_norm.head(5)

,a,q,data_arc,n_obs_used,H,albedo,moid
0,-0.029792,0.302503,-0.012622,0.588733,-8.368553,-0.361170,0.341052
1,-0.027624,-0.520575,10.305415,13.457097,-7.810651,-0.260805,-0.364798
2,-0.095484,-0.812229,10.364889,11.075212,-6.963205,0.770221,-0.752728
3,-0.297605,-0.485614,2.500587,14.892070,-8.467421,2.675338,-0.547849
4,-0.157815,-0.620434,8.861286,3.783483,-5.889774,1.317669,-0.632952


#### Test Data

In [11]:
xte_norm=sd.transform(xte[['a', 'q', 'data_arc', 'n_obs_used', 'H',
       'albedo', 'moid']])
xte_norm=pd.DataFrame(data=xte_norm,columns=['a', 'q', 'data_arc', 'n_obs_used', 'H',
       'albedo', 'moid'])
xte_norm.head(5)

,a,q,data_arc,n_obs_used,H,albedo,moid
0,-0.105408,-0.397108,5.067909,2.492866,-2.111580,-0.47066,-0.399530
1,-0.133678,-1.671668,5.466519,2.599415,-2.450558,-0.47066,-1.553292
2,-0.115431,-2.335321,4.968947,2.003084,0.148275,-0.47066,-2.375477
3,-0.375146,-1.079947,4.909473,2.618319,-0.981652,-0.47066,-1.129684
4,-0.351346,-1.164690,4.660362,1.717808,-0.981652,-0.47066,-1.214086


#### Encoding Numerical and Categorical Features

* Train Data

In [12]:
xtr=xtr_norm[['a','q','data_arc','n_obs_used','H','albedo','moid']]
xtr['neo']=xtr_neo_encode.values
xtr['pha']=xtr_pha_encode.values
xtr.head(5)

,a,q,data_arc,n_obs_used,H,albedo,moid,neo,pha
0,-0.029792,0.302503,-0.012622,0.588733,-8.368553,-0.361170,0.341052,0,0
1,-0.027624,-0.520575,10.305415,13.457097,-7.810651,-0.260805,-0.364798,0,0
2,-0.095484,-0.812229,10.364889,11.075212,-6.963205,0.770221,-0.752728,0,0
3,-0.297605,-0.485614,2.500587,14.892070,-8.467421,2.675338,-0.547849,0,0
4,-0.157815,-0.620434,8.861286,3.783483,-5.889774,1.317669,-0.632952,0,0


* Test Data

In [13]:
xte=xte_norm[['a','q','data_arc','n_obs_used','H','albedo','moid']]
xte['neo']=xte_neo_encode.values
xte['pha']=xte_pha_encode.values
xte.head(5)

,a,q,data_arc,n_obs_used,H,albedo,moid,neo,pha
0,-0.105408,-0.397108,5.067909,2.492866,-2.111580,-0.47066,-0.399530,0,0
1,-0.133678,-1.671668,5.466519,2.599415,-2.450558,-0.47066,-1.553292,0,0
2,-0.115431,-2.335321,4.968947,2.003084,0.148275,-0.47066,-2.375477,1,0
3,-0.375146,-1.079947,4.909473,2.618319,-0.981652,-0.47066,-1.129684,0,0
4,-0.351346,-1.164690,4.660362,1.717808,-0.981652,-0.47066,-1.214086,0,0


#### Building Models

### Using R Squared as performance metric for the problem

* Linear Regression

In [14]:
# Assumptions of multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

print(calc_vif(xtr))

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


    variables         VIF
0           a    1.160467
1           q  217.798754
2    data_arc    2.523042
3  n_obs_used    5.915914
4           H    6.203478
5      albedo    1.364416
6        moid  222.042839
7         neo    1.859653
8         pha    1.352095


After looking at the above observation, it can be concluded that:
1. ‘q’ and ‘moid’ have a high VIF value, meaning they can be predicted by other independent variables in the dataset.
2. VIF is preferred as it can show the correlation of a variable with a group of other variables.
3. We can drop higher correlated features.

In [15]:
xtr1=xtr.copy() # Will have highest correlated feature dropped
xte1=xte.copy() # Will have highest correlated feature dropped

#### Training the model for dropped correlated feature

In [16]:
xtr1.drop(columns=['moid'],axis=1,inplace=True)
xte1.drop(columns=['moid'],axis=1,inplace=True)

In [17]:
print(calc_vif(xtr1))

    variables       VIF
0           a  1.158789
1           q  2.263614
2    data_arc  2.511184
3  n_obs_used  5.576538
4           H  5.757707
5      albedo  1.364138
6         neo  1.453489
7         pha  1.351712


In [18]:
xtr1,xcv1,ytr1,ycv1=train_test_split(xtr1,ytr,test_size=0.2)

In [19]:
model=LinearRegression()
model.fit(xtr1,ytr1)
ypred=model.predict(xte1)

In [20]:
print("R^2 of the train model is : ",model.score(xtr1,ytr1))

R^2 of the train model is :  0.4328952103757191


* Ridge Regression

In [21]:
from numpy import arange
from pandas import read_csv
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge
# load the dataset
# define model
model = Ridge()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = [0.001,0.01,0.1,1,10]
# define search
search = GridSearchCV(model, grid, scoring='r2', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(xtr1, ytr1)
# summarize
print('R Squared Error: %.3f' % results.best_score_)
print('Best Param: %s' % results.best_params_)

R Squared Error: 0.446
Best Param: {'alpha': 10}


* Lasso Regression

In [22]:
# define model
model = Lasso()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = [0.001,0.01,0.1,1,10]
# define search
search = GridSearchCV(model, grid, scoring='r2', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(xtr1, ytr1)
# summarize
print('R Squared : %.3f' % results.best_score_)
print('Best Params : %s' % results.best_params_)

R Squared : 0.446
Best Params : {'alpha': 0.001}


* Elastic Net

In [23]:
# define model
model = ElasticNet()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = [0.001,0.01,0.1,1,10]
grid['max_iter']=[100,200]
grid['l1_ratio']=arange(0, 1, 0.1)
search = GridSearchCV(model, grid, scoring='r2', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(xtr1, ytr1)
# summarize
print('R Squared : %.3f' % results.best_score_)
print('Best Params : %s' % results.best_params_)

R Squared : 0.446
Best Params : {'alpha': 0.001, 'l1_ratio': 0.9, 'max_iter': 100}


* Adaboost Regressor

In [29]:
# Splitting the dataset
xtr,xcv,ytr,ycv=train_test_split(xtr,ytr,test_size=0.2)

In [32]:
from sklearn.model_selection import RepeatedKFold,KFold,cross_val_score
from sklearn.ensemble import AdaBoostRegressor
ada=AdaBoostRegressor()
# evaluate the model
grid = dict()
grid['n_estimators'] = [100, 250, 500]
grid['learning_rate'] = [0.0001, 0.001]
# define the grid search procedure
grid_search = GridSearchCV(estimator=ada, param_grid=grid, n_jobs=-1, cv=10, scoring='r2',verbose=2)
# execute the grid search
grid_result = grid_search.fit(xtr, ytr)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 10 folds for each of 6 candidates, totalling 60 fits
Best: 0.853050 using {'learning_rate': 0.001, 'n_estimators': 500}


* Random Forest Regressor

In [31]:
param_grid = {
    'max_depth': [3,5],
    'n_estimators': [100,250,500]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf,scoring="r2", param_grid = param_grid, 
                          cv = 10 , n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(xtr,ytr)
print('R Squared : %.3f' % grid_search.best_score_)
print('Best Params : %s' % grid_search.best_params_)

Fitting 10 folds for each of 6 candidates, totalling 60 fits
R Squared : 0.953
Best Params : {'max_depth': 5, 'n_estimators': 100}


#### XGBoost Regressor

In [30]:
param_grid = {
    'max_depth': [3,5],
    'n_estimators': [100,250,500]
}
# Create a based model
xgb = XGBRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgb,scoring="r2", param_grid = param_grid, 
                          cv = 10 , n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(xtr,ytr)
print('R Squared : %.3f' % grid_search.best_score_)
print('Best Params : %s' % grid_search.best_params_)

Fitting 10 folds for each of 6 candidates, totalling 60 fits
[07:48:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
R Squared : 0.963
Best Params : {'max_depth': 3, 'n_estimators': 250}


## Negative Mean Absolute Error

* This is the negative value of mean absolute error.
* The best value is 0.
* Value closer to 0 indicates less error.
* Value farther from 0 indicates higher error.   

#### Ridge Regression

In [24]:
# define model
model = Ridge()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = [0.001,0.01,0.1,1,10]
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error',cv=cv, n_jobs=-1)
# perform the search
results = search.fit(xtr1, ytr1)
# summarize
print('Negative Mean Absolute Error: %.3f' % results.best_score_)
print('Best Param: %s' % results.best_params_)

Negative Mean Absolute Error: -2.688
Best Param: {'alpha': 10}


#### Lasso Regression

In [25]:
# define model
model = Lasso()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = [0.001,0.01,0.1,1,10]
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error',cv=cv, n_jobs=-1)
# perform the search
results = search.fit(xtr1, ytr1)
# summarize
print('Negative Mean Absolute Error: %.3f' % results.best_score_)
print('Best Param: %s' % results.best_params_)

Negative Mean Absolute Error: -2.146
Best Param: {'alpha': 1}


#### ElasticNet Regression

In [27]:
# define model
model = ElasticNet()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = [0.001,0.01,0.1,1,10]
grid['max_iter']=[100,200]
grid['l1_ratio']=arange(0, 1, 0.1)
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(xtr1, ytr1)
# summarize
print('R Squared : %.3f' % results.best_score_)
print('Best Params : %s' % results.best_params_)

R Squared : -1.983
Best Params : {'alpha': 1, 'l1_ratio': 0.0, 'max_iter': 100}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.714e+06, tolerance: 1.019e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


#### AdaBoost Regressor

In [ ]:
from sklearn.model_selection import RepeatedKFold,KFold,cross_val_score
from sklearn.ensemble import AdaBoostRegressor
ada=AdaBoostRegressor()
# evaluate the model
grid = dict()
grid['n_estimators'] = [100, 250, 500]
grid['learning_rate'] = [0.0001, 0.001]
# define the grid search procedure
grid_search = GridSearchCV(estimator=ada, param_grid=grid, n_jobs=-1, cv=10, scoring='neg_mean_absolute_error',verbose=2)
# execute the grid search
grid_result = grid_search.fit(xtr, ytr)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 10 folds for each of 6 candidates, totalling 60 fits
Best: -1.803316 using {'learning_rate': 0.0001, 'n_estimators': 250}


#### Random Forest Regressor

In [ ]:
param_grid = {
    'max_depth': [3,5],
    'n_estimators': [100,250,500]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf,scoring="neg_mean_absolute_error", param_grid = param_grid, 
                          cv = 10 , n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(xtr,ytr)
print('R Squared : %.3f' % grid_search.best_score_)
print('Best Params : %s' % grid_search.best_params_)

Fitting 10 folds for each of 6 candidates, totalling 60 fits
R Squared : -0.914
Best Params : {'max_depth': 5, 'n_estimators': 500}


#### XGBoost Regressor

In [ ]:
param_grid = {
    'max_depth': [3,5],
    'n_estimators': [100,250,500]
}
# Create a based model
xgb = XGBRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgb,scoring="neg_mean_absolute_error", param_grid = param_grid, 
                          cv = 10 , n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(xtr,ytr)
print('R Squared : %.3f' % grid_search.best_score_)
print('Best Params : %s' % grid_search.best_params_)

Fitting 10 folds for each of 6 candidates, totalling 60 fits
[07:51:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
R Squared : -0.405
Best Params : {'max_depth': 5, 'n_estimators': 500}


### Pretty Table Observations

#### For R Squared Error

In [33]:
from prettytable import PrettyTable
tab = PrettyTable(["Sr No", "Model", "Best Hyperparameters", "Best Score(R Squared)"])
# Add rows
tab.add_row(["1.", "Linear Regression", "-", "0.4329"])
tab.add_row(["2.", "Ridge Regression", "{'alpha' : 10}", "0.446"])
tab.add_row(["3.", "Lasso Regression", "{'alpha' : 0.001}", "0.446"])
tab.add_row(["4.", "ElasticNet Regression", "{'alpha' : 0.001,'l1_ratio': 0.9, 'max_iter': 100}", "0.446"])
tab.add_row(["5.","Adaboost Regressor","{'learning_rate': 0.0001, 'n_estimators': 250}","0.853"])
tab.add_row(["6.","Random Forest Regressor","{'max_depth': 5, 'n_estimators': 100}","0.953"])
tab.add_row(["7.","XGBoost Regressor","{'max_depth': 3, 'n_estimators': 250}","0.963"])
print(tab)

+-------+-------------------------+----------------------------------------------------+-----------------------+
| Sr No |          Model          |                Best Hyperparameters                | Best Score(R Squared) |
+-------+-------------------------+----------------------------------------------------+-----------------------+
|   1.  |    Linear Regression    |                         -                          |         0.4329        |
|   2.  |     Ridge Regression    |                   {'alpha' : 10}                   |         0.446         |
|   3.  |     Lasso Regression    |                 {'alpha' : 0.001}                  |         0.446         |
|   4.  |  ElasticNet Regression  | {'alpha' : 0.001,'l1_ratio': 0.9, 'max_iter': 100} |         0.446         |
|   5.  |    Adaboost Regressor   |   {'learning_rate': 0.0001, 'n_estimators': 250}   |         0.853         |
|   6.  | Random Forest Regressor |       {'max_depth': 5, 'n_estimators': 100}        |        

#### Negative Mean Absolute Error

In [34]:
from prettytable import PrettyTable
tab = PrettyTable(["Sr No", "Model", "Best Hyperparameters", "Negative Mean Absolute Error"])
# Add rows
tab.add_row(["1.", "Linear Regression", "-", "-"])
tab.add_row(["2.", "Ridge Regression", "{'alpha' : 10}", "-2.688"])
tab.add_row(["3.", "Lasso Regression", "{'alpha' : 1}", "-2.146"])
tab.add_row(["4.", "ElasticNet Regression", "{'alpha': 1, 'l1_ratio': 0.0, 'max_iter': 100}", "-1.983"])
tab.add_row(["5.","Adaboost Regressor","{'learning_rate': 0.0001, 'n_estimators': 250}","-1.803"])
tab.add_row(["6.","Random Forest Regressor","{'max_depth': 5, 'n_estimators': 500}","-0.914"])
tab.add_row(["7.","XGBoost Regressor","{'max_depth': 5, 'n_estimators': 500}","-0.405"])
print(tab)

+-------+-------------------------+------------------------------------------------+------------------------------+
| Sr No |          Model          |              Best Hyperparameters              | Negative Mean Absolute Error |
+-------+-------------------------+------------------------------------------------+------------------------------+
|   1.  |    Linear Regression    |                       -                        |              -               |
|   2.  |     Ridge Regression    |                 {'alpha' : 10}                 |            -2.688            |
|   3.  |     Lasso Regression    |                 {'alpha' : 1}                  |            -2.146            |
|   4.  |  ElasticNet Regression  | {'alpha': 1, 'l1_ratio': 0.0, 'max_iter': 100} |            -1.983            |
|   5.  |    Adaboost Regressor   | {'learning_rate': 0.0001, 'n_estimators': 250} |            -1.803            |
|   6.  | Random Forest Regressor |     {'max_depth': 5, 'n_estimators':

### Conclusion:

<b>For R squared:</b>
1. After lots of model training, It can be observed that XGBoost Regressor performs best out of all models.

2. Lasso, Ridge and Elastic Net Regression has almost similar performance.

3. Out of all Models, Linear Regression performs poorly.

4. R^2 Error is taken as performance metric in this regression problem.

5. All of the ensembles perform very well.

6. Higher values of R-square determines the less difference between the predicted values and actual values and hence represents a good model.



<b> For Negative Mean Absolute Error : </b>

1. After lots of model training, It can be observed that XGBoost Regressor performs best in terms of negative mean absolute error out of all models.

2. Lasso Performs better than Ridge whereas ElasticNet gives out the best performance out of all Linear Regression models.

3. Negative Mean Absolute Error is taken as performance metric in this regression problem.

4. All of the ensembles perform very well.

5. Higher values of Negative Mean Absolute Error determines the less difference between the predicted values and actual values and hence represents a good model.
